<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [2]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagement/master/financial_models/Opportunities/Monitor/screener_summary.csv'
df = pd.read_csv(summary_url)
df = df.set_index('Ticker')
display(df)

,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Last_fy,TotalAssets,...,OtherShortTermInvestments,InvestmentProperties,LongTermEquityInvestment,InvestmentinFinancialAssets,NetPPE,TotalRevenue,CostOfRevenue,SellingGeneralAndAdministration,InterestExpense,NetIncomeCommonStockholders
Ticker,,,,,,,,,,,,,,,,,,,,,
0018.HK,ORIENTAL E H,HKG,0.58,HKD,2397920000,HKD,1.00000,0.000000,1648684800000,1.924173e+09,...,0.000000e+00,2.886760e+08,0.000000e+00,8.820000e+06,432643000.0,7.357820e+08,7.149900e+07,0.0,276000.0,1.667440e+08
0026.HK,CHINA MOTOR BUS,HKG,71.20,HKD,45308100,HKD,1.00000,3.167535,1656547200000,7.962093e+09,...,1.405278e+09,2.364393e+09,3.990683e+09,1.461800e+07,96610000.0,7.298300e+07,6.362000e+06,8850000.0,0.0,7.234700e+07
0029.HK,DYNAMIC HOLD,HKG,9.50,HKD,237704000,HKD,1.00000,0.015002,1656547200000,2.685457e+09,...,9.837400e+07,2.005063e+09,9.116300e+07,0.000000e+00,5135000.0,8.689200e+07,2.496500e+07,43891000.0,1691000.0,3.115200e+07
0032.HK,CROSS-HAR(HOLD),HKG,10.90,HKD,372688000,HKD,1.00000,0.420510,1640908800000,8.021072e+09,...,1.310188e+09,0.000000e+00,5.797470e+08,3.099484e+09,379600000.0,6.996660e+08,2.287760e+08,221518000.0,2672000.0,-4.924500e+07
0050.HK,HK FERRY (HOLD),HKG,8.25,HKD,356273984,HKD,1.00000,0.000000,1640908800000,6.340123e+09,...,0.000000e+00,2.272505e+09,5.021670e+08,1.346320e+08,121116000.0,2.441970e+08,1.646220e+08,57409000.0,122000.0,1.182490e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899.HK,CLOUD MUSIC,HKG,85.80,HKD,209600992,CNY,1.13078,0.000000,1640908800000,9.889339e+09,...,5.872475e+09,0.000000e+00,9.039300e+07,0.000000e+00,57627000.0,6.997622e+09,6.854948e+09,695528000.0,0.0,-2.056092e+09
9982.HK,CENTRALCHINA MT,HKG,0.68,HKD,3282660096,CNY,1.13078,0.071836,1640908800000,2.723630e+09,...,0.000000e+00,0.000000e+00,2.230000e+06,4.182000e+06,14016000.0,1.301449e+09,0.000000e+00,19711000.0,413000.0,7.701550e+08
9983.HK,CC NEW LIFE,HKG,3.30,HKD,1278729984,CNY,1.13078,0.266540,1640908800000,5.043390e+09,...,3.829800e+07,0.000000e+00,5.809000e+06,1.160800e+07,81257000.0,3.598918e+09,2.415696e+09,286758000.0,3453000.0,6.195590e+08


In [3]:
# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price * df['Fx_rate']
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
# more easily realizable non-operating assets
monetary_assets = df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
total_nonop_assets = monetary_assets + fixed_nonop_assets
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents'] - total_nonop_assets
# liquidity_coverage_ratio
lcr = monetary_assets / df['CurrentLiabilities']
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
# dividend rate
dividend_rate = df['Dividend'] / df['Price']

# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [23]:
# common fitlering conditions
condition_1 = df['CashAndCashEquivalents'] > df['CurrentDebtAndCapitalLeaseObligation']
condition_2 = lcr >= 1.2
condition_3 = (total_debt / df['CurrentAssets']) < 1.1

In [35]:
# 1st set of conditions
condition_4 = capitalization_price > 10000000000

In [36]:
# 2nd set of conditions
condition_5 = capitalization_price <= 10000000000
condition_6 = dividend_rate > 0.01

#Step 3. Output

Filter the dataset using the above conditions

In [40]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['Dividend rate'] = dividend_rate
common_df = common_df.loc[condition_1 & condition_2 & condition_3]
print(common_df.to_string())

                    Name Exchange    Price Price_currency       Shares Reporting_Currency   Fx_rate  Dividend        Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  CostOfRevenue  SellingGeneralAndAdministration  InterestExpense  NetIncomeCommonStockholders            EV  Dividend rate
Ticker                                                                                                                                                                                                                                                                                                                                                             

In [41]:
# fitlered by 1st set of conditions
df_1 = common_df
df_1 = df_1.loc[condition_4]
print(df_1.to_string())

                Name Exchange  Price Price_currency      Shares Reporting_Currency  Fx_rate  Dividend        Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  CostOfRevenue  SellingGeneralAndAdministration  InterestExpense  NetIncomeCommonStockholders            EV  Dividend rate
Ticker                                                                                                                                                                                                                                                                                                                                                                     

In [42]:
# filtered by 2nd set of conditions
df_2 = common_df
df_2 = df_2.loc[condition_5 & condition_6]
print(df_2.to_string())

                    Name Exchange  Price Price_currency      Shares Reporting_Currency  Fx_rate  Dividend        Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  CostOfRevenue  SellingGeneralAndAdministration  InterestExpense  NetIncomeCommonStockholders            EV  Dividend rate
Ticker                                                                                                                                                                                                                                                                                                                                                                 

In [44]:
# combine the results
result_set = pd.concat([df_1, df_2])
result_set = result_set.groupby("Ticker").first()
result_set = result_set.sort_values(by=['EV', 'Dividend rate'], ascending=[True, False]).reset_index()
print(result_set.to_string())

     Ticker             Name Exchange  Price Price_currency      Shares Reporting_Currency  Fx_rate  Dividend        Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  CostOfRevenue  SellingGeneralAndAdministration  InterestExpense  NetIncomeCommonStockholders            EV  Dividend rate
0   3983.HK   CHINA BLUECHEM      HKG   1.89            HKD  1771000064                CNY  1.13078  0.208244  1640908800000  2.203146e+10   1.394292e+10        3.507177e+09                           645852000.0                     12972000.0                           9.941050e+08                      12401000.0                      1.708854e+10        